<a href="https://colab.research.google.com/github/sauravsingla/Multi-Class-Sentiment-Classification-/blob/main/twitter_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.layers import Bidirectional, GRU, GlobalMaxPool1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import time
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import KernelPCA
from pandas import DataFrame
from sklearn.model_selection import train_test_split
import re
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("/content/drive/My Drive/NLP/text_emotion.csv")
df = df.drop(["tweet_id","author"],axis = 1)
print(df.head())

    sentiment                                            content
0       empty  @tiffanylue i know  i was listenin to bad habi...
1     sadness  Layin n bed with a headache  ughhhh...waitin o...
2     sadness                Funeral ceremony...gloomy friday...
3  enthusiasm               wants to hang out with friends SOON!
4     neutral  @dannycastillo We want to trade with someone w...


In [ ]:
df.drop(df[df['sentiment'] == 'empty'].index, inplace = True)
df.drop(df[df['sentiment'] == 'enthusiasm'].index, inplace = True)
df['target'] = df['sentiment'].map({'sadness':0, 'boredom':1,'neutral':2,'worry':3,'surprise':4,'love':5,'fun':6,'hate':7,'happiness':8,'anger':9,'relief':10})
df = df.drop(["sentiment"],axis=1)
print(df.head())

                                             content  target
1  Layin n bed with a headache  ughhhh...waitin o...       0
2                Funeral ceremony...gloomy friday...       0
4  @dannycastillo We want to trade with someone w...       2
5  Re-pinging @ghostridah14: why didn't you go to...       3
6  I should be sleep, but im not! thinking about ...       0


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"didn't", "did not", text)
    return text

In [ ]:
cleaned_text = []
for text in df['content']:
    cleaned_text.append(clean_text(text))
df['clean'] = cleaned_text
df.head()

,content,target,clean
1,Layin n bed with a headache ughhhh...waitin o...,0,layin n bed with a headache ughhhh...waitin o...
2,Funeral ceremony...gloomy friday...,0,funeral ceremony...gloomy friday...
4,@dannycastillo We want to trade with someone w...,2,@dannycastillo we want to trade with someone w...
5,Re-pinging @ghostridah14: why didn't you go to...,3,re-pinging @ghostridah14: why did not you go t...
6,"I should be sleep, but im not! thinking about ...",0,"i should be sleep, but im not! thinking about ..."


In [ ]:
tw = []
for j in df['clean']:
  tweets = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(j)).split())
  tw.append(tweets)
df['clean'] = tw
df.head()

,content,target,clean
1,Layin n bed with a headache ughhhh...waitin o...,0,layin n bed with a headache ughhhh waitin on y...
2,Funeral ceremony...gloomy friday...,0,funeral ceremony gloomy friday
4,@dannycastillo We want to trade with someone w...,2,we want to trade with someone who has houston ...
5,Re-pinging @ghostridah14: why didn't you go to...,3,re pinging why did not you go to prom bc my bf...
6,"I should be sleep, but im not! thinking about ...",0,i should be sleep but im not thinking about an...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
output = []
for sentence in df["clean"]:
    temp_list = []
    for word in sentence.split():
      if len(word) > 2 not in stopwords:
        temp_list.append(word)
    output.append(' '.join(temp_list))
    
df["texts"] = output
df = df.drop(['content','clean'], axis = 1)
df.head()

,target,texts
1,0,layin bed with headache ughhhh waitin your call
2,0,funeral ceremony gloomy friday
4,2,want trade with someone who has houston ticket...
5,3,pinging why did not you prom did not like friends
6,0,should sleep but not thinking about old friend...


In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

def lemmatize_text(texts):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(texts)]


df['text_lemmatized'] = df.texts.apply(lemmatize_text)

sc = [[' '.join(i)] for i in df['text_lemmatized']]
lis = []
for i in sc:
    abc = i[0]
    lis.append(abc)

df['lem'] = lis
df = df.drop(['text_lemmatized','texts'], axis =1)
df.head()

,target,lem
1,0,layin bed with headache ughhhh waitin your call
2,0,funeral ceremony gloomy friday
4,2,want trade with someone who ha houston ticket ...
5,3,pinging why did not you prom did not like friend
6,0,should sleep but not thinking about old friend...


In [ ]:
X = df["lem"]
Y = df["target"]
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size = 0.3)

1          layin bed with headache ughhhh waitin your call
2                           funeral ceremony gloomy friday
4        want trade with someone who ha houston ticket ...
5         pinging why did not you prom did not like friend
6        should sleep but not thinking about old friend...
                               ...                        
39995                                                     
39996                            happy mother day all love
39997    happy mother day all the mommy out there you w...
39998    wassup beautiful follow peep out new hit singl...
39999    bullet train from tokyo the and have been visi...
Name: lem, Length: 38414, dtype: object
1        0
2        0
4        2
5        3
6        0
        ..
39995    2
39996    5
39997    5
39998    8
39999    5
Name: target, Length: 38414, dtype: int64


In [ ]:
## word2vec

class Word2VecVectorizer:
  def __init__(self):
    print("Loading in word vectors...")
    self.word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/NLP/GoogleNews-vectors-negative300.bin', binary=True)
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('king')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

vectorizer = Word2VecVectorizer()
Xtrain = vectorizer.fit_transform(Xtrain)

Xtest = vectorizer.transform(Xtest)

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 154 / 26889
Numer of samples with no words found: 57 / 11525


In [ ]:
# create the model, train it, print scores

model = RandomForestClassifier(n_estimators=200)
model.fit(Xtrain, Ytrain)
print("train score:", model.score(Xtrain, Ytrain))
print("test score:", model.score(Xtest, Ytest))


y_pred = model.predict(Xtest)
cnfv=confusion_matrix(Ytest,y_pred)
print(cnfv)


print(classification_report(Ytest, y_pred))

train score: 0.9823719736695303
test score: 0.34004338394793926
[[ 122    0  424  916    2   26    0    6   64    1    0]
 [   3    0   22   25    0    0    0    0    3    0    0]
 [  39    1 1579  701    7   66    6    0  160    0    3]
 [  94    0  863 1481    0   60    0    3  104    0    4]
 [   7    0  252  268    1   25    0    1   67    0    0]
 [  26    0  342  258    3  325    2    0  216    0    0]
 [   6    0  222  186    0   12    1    0  106    0    1]
 [  18    0  122  269    0    3    0    6    9    0    0]
 [  14    0  583  389    0  120    4    0  404    0    2]
 [   1    0    9   19    0    1    0    0    4    0    0]
 [   4    0  187  166    0   15    0    0   64    0    0]]
              precision    recall  f1-score   support

           0       0.37      0.08      0.13      1561
           1       0.00      0.00      0.00        53
           2       0.34      0.62      0.44      2562
           3       0.32      0.57      0.41      2609
           4       0.08   